# 10 · Monitoramento e Drift Detection
## Sistema de Vigilância Contínua de Modelos

<div align="center">

```
┌─────────────────────────────────────────────────────────────┐
│   MODEL MONITORING SYSTEM - DRIFT DETECTION v1.0          │
└─────────────────────────────────────────────────────────────┘
```

![Status](https://img.shields.io/badge/Status-Monitoring-blue)
![Type](https://img.shields.io/badge/Type-Production%20Health-success)
![Priority](https://img.shields.io/badge/Priority-HIGH-orange)

</div>

---

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**OBJETIVO**

Implementar sistema de monitoramento contínuo para detectar degradação de performance, drift em features e problemas de qualidade que comprometam a eficácia investigativa do modelo em produção.

</div>

### TIPOS DE DRIFT DETECTADOS

<table>
<tr><th>Tipo de Drift</th><th>Descrição</th><th>Threshold PSI</th><th>Ação</th></tr>
<tr><td><b>Score Drift</b></td><td>Mudança na distribuição de scores preditos</td><td>> 0.2</td><td>Investigar causas</td></tr>
<tr><td><b>Feature Drift</b></td><td>Mudança nas distribuições de features</td><td>> 0.25</td><td>Retreinar modelo</td></tr>
<tr><td><b>Concept Drift</b></td><td>Mudança na relação feature-target</td><td>Performance drop > 10%</td><td>Redesenhar features</td></tr>
<tr><td><b>Data Quality</b></td><td>Missing values, outliers, schema changes</td><td>Variável</td><td>Pipeline fix</td></tr>
</table>

### METODOLOGIA DE MONITORAMENTO

```
REFERÊNCIA (Treino)              PRODUÇÃO (Novo)
┌──────────────────┐            ┌──────────────────┐
│  Feature         │            │  Feature         │
│  Distributions   │            │  Distributions   │
│  (Baseline)      │            │  (Current)       │
└────────┬─────────┘            └────────┬─────────┘
         │                               │
         └───────────────┬───────────────┘
                         │
                    ┌────▼─────┐
                    │   PSI    │
                    │ Calculation│
                    └────┬─────┘
                         │
         ┌───────────────┼───────────────┐
         │               │               │
    ┌────▼────┐    ┌────▼────┐    ┌────▼────┐
    │PSI<0.1  │    │0.1≤PSI  │    │PSI≥0.25 │
    │STABLE   │    │<0.25    │    │CRITICAL │
    │[OK]     │    │MONITOR  │    │ACTION   │
    └─────────┘    └─────────┘    └─────────┘
```

<div style="background-color: #1a2332; border-left: 4px solid #3b82f6; padding: 15px; border-radius: 4px; margin-top: 15px;">

**INTERPRETAÇÃO PSI (POPULATION STABILITY INDEX)**

**THRESHOLDS PADRÃO:**
- **PSI < 0.1**: Sem drift detectável ✅ Estável
- **PSI 0.1-0.2**: Pequena mudança ⚠️ Monitorar
- **PSI 0.2-0.25**: Mudança moderada ⚠️ Investigar
- **PSI ≥ 0.25**: Mudança significativa 🔴 Ação necessária

**FÓRMULA:**
```
PSI = Σ (% atual - % esperado) × ln(% atual / % esperado)
```

</div>

### PIPELINE DE MONITORAMENTO

```
┌──────────────────┐
│ 1. Score Drift   │──► PSI em distribuição de scores
└────────┬─────────┘
         │
┌────────▼─────────┐
│ 2. Feature Drift │──► PSI por feature (top alertas)
└────────┬─────────┘
         │
┌────────▼─────────┐
│ 3. Performance   │──► Métricas operacionais
│    Monitoring    │
└────────┬─────────┘
         │
┌────────▼─────────┐
│ 4. Alert System  │──► Notificações + Reports
└──────────────────┘
```

<div style="background-color: #1a2332; border-left: 4px solid #3b82f6; padding: 15px; border-radius: 4px; margin-top: 15px;">

**NOTA TÉCNICA**

Este notebook simula monitoramento em batch. Para produção real, integrar com pipelines de streaming (Kafka/Kinesis).

</div>

---

In [1]:
import json
import pickle
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

PROJECT_ROOT = Path("..").resolve()
UTILS_PATH = PROJECT_ROOT / "utils"
if str(UTILS_PATH) not in sys.path:
    sys.path.append(str(UTILS_PATH))

from monitoring import feature_shift_table, population_stability_index, score_shift_report

print("Monitoring utilities imported successfully.")


Monitoring utilities imported successfully.


In [3]:
# Paths and references
DATA_PATH = PROJECT_ROOT / "data"
ARTIFACT_PATH = PROJECT_ROOT / "artifacts"

ensemble_file = ARTIFACT_PATH / "risk_scores_ensemble.csv"
fallback_file = ARTIFACT_PATH / "risk_scores_test.csv"

if ensemble_file.exists():
    risk_ref = pd.read_csv(ensemble_file)
    print("Reference data: ensemble scores.")
elif fallback_file.exists():
    risk_ref = pd.read_csv(fallback_file)
    print("Reference data: base risk scores (fallback).")
else:
    # Generate synthetic reference data for demonstration
    print("Warning: No risk score artifacts found. Generating synthetic reference data for monitoring demo.")
    n_samples = 1000
    rng_temp = np.random.default_rng(42)
    risk_ref = pd.DataFrame({
        'ensemble_score': rng_temp.beta(2, 5, n_samples),
        'score_supervised': rng_temp.beta(2, 5, n_samples),
        'score_final': rng_temp.beta(2, 5, n_samples)
    })
    print("Generated synthetic reference data for demonstration purposes.")

print(f"Reference batch: {risk_ref.shape[0]} rows, columns: {list(risk_ref.columns)}")

rng = np.random.default_rng(42)
risk_new = risk_ref.sample(frac=0.6, random_state=42).copy()

if "ensemble_score" in risk_new.columns:
    drift_factor = 1 + rng.normal(0, 0.08, len(risk_new))
    risk_new["ensemble_score"] = np.clip(risk_new["ensemble_score"] * drift_factor, 0, 1)
    score_col = "ensemble_score"
elif "score_final" in risk_new.columns:
    drift_factor = 1 + rng.normal(0, 0.06, len(risk_new))
    risk_new["score_final"] = np.clip(risk_new["score_final"] * drift_factor, 0, 1)
    score_col = "score_final"
else:
    drift_factor = 1 + rng.normal(0, 0.05, len(risk_new))
    risk_new["score_supervised"] = np.clip(risk_new["score_supervised"] * drift_factor, 0, 1)
    score_col = "score_supervised"

print(f"Simulated production batch: {risk_new.shape[0]} rows.")
print(f"Monitoring score column: {score_col}")


Generated synthetic reference data for demonstration purposes.
Reference batch: 1000 rows, columns: ['ensemble_score', 'score_supervised', 'score_final']
Simulated production batch: 600 rows.
Monitoring score column: ensemble_score


In [4]:
# Score drift analysis
print(f"Assessing drift for score column: {score_col}")

ref_scores = risk_ref[score_col].dropna()
new_scores = risk_new[score_col].dropna()

score_report = score_shift_report(ref_scores, new_scores)

psi_score = score_report["psi_scores"]
if psi_score < 0.1:
    psi_interpretation = "Stable (< 0.1)"
elif psi_score < 0.2:
    psi_interpretation = "Minor change (0.1–0.2)"
else:
    psi_interpretation = "Significant change (> 0.2)"

print(f"PSI score: {psi_score:.4f} — {psi_interpretation}")
print(f"Mean shift: {score_report['mean_ref']:.4f} → {score_report['mean_new']:.4f}")
print(f"Std shift:  {score_report['std_ref']:.4f} → {score_report['std_new']:.4f}")

score_drift_df = pd.DataFrame(
    [
        {
            "metric": "score_drift",
            "score_column": score_col,
            "psi": psi_score,
            "interpretation": psi_interpretation,
            **score_report,
        }
    ]
)

score_drift_df


Assessing drift for score column: ensemble_score
PSI score: 0.0148 — Stable (< 0.1)
Mean shift: 0.2923 → 0.2983
Std shift:  0.1571 → 0.1675


,metric,score_column,psi,interpretation,psi_scores,mean_ref,mean_new,std_ref,std_new
0,score_drift,ensemble_score,0.014755,Stable (< 0.1),0.014755,0.292305,0.298347,0.157051,0.167464


In [5]:
# Feature drift analysis (PSI by feature)
print("Evaluating feature-level drift (PSI)...")

feature_files = [
    DATA_PATH / "X_test_graph.csv",
    DATA_PATH / "X_test_engineered.csv",
    DATA_PATH / "X_test.csv",
]

X_ref = None
for file_path in feature_files:
    if file_path.exists():
        X_ref = pd.read_csv(file_path)
        print(f"Using feature baseline from {file_path.name}.")
        break

if X_ref is not None:
    X_new = X_ref.sample(frac=0.7, random_state=7).copy()

    numeric_cols = X_new.select_dtypes(include=["number"]).columns
    drift_cols = rng.choice(numeric_cols, size=min(10, len(numeric_cols)), replace=False)

    for col in drift_cols:
        drift_intensity = rng.uniform(0.02, 0.12)
        drift_bias = rng.uniform(-0.05, 0.05)

        original_values = X_new[col].values
        noise = rng.normal(0, drift_intensity, len(X_new))
        X_new[col] = original_values * (1 + noise + drift_bias)

    print(f"Applied drift to {len(drift_cols)} features: {list(drift_cols[:5])}...")

    feature_drift = feature_shift_table(X_ref, X_new)

    if len(feature_drift) > 0:
        feature_drift["interpretation"] = feature_drift["psi"].apply(
            lambda x: "Stable" if x < 0.1 else "Minor change" if x < 0.2 else "Significant"
        )

        print("\nTop 10 features with highest PSI:")
        top_drift = feature_drift.head(10)
        for _, row in top_drift.iterrows():
            print(f"  {row['feature']}: PSI={row['psi']:.4f} ({row['interpretation']})")

        feature_drift.head(10)
    else:
        print("No numeric features available for drift analysis.")
        feature_drift = pd.DataFrame()
else:
    print("Feature baseline not found; skipping feature drift computation.")
    feature_drift = pd.DataFrame()


Evaluating feature-level drift (PSI)...
Using feature baseline from X_test_graph.csv.
Applied drift to 10 features: ['Account_freq', 'bank_interaction', 'To Bank_freq', 'Amount Paid', 'Dest Account']...

Top 10 features with highest PSI:
  From Bank: PSI=0.0829 (Stable)
  To Bank_freq: PSI=0.0147 (Stable)
  From Bank_freq: PSI=0.0103 (Stable)
  To Bank: PSI=0.0089 (Stable)
  Amount Received: PSI=0.0066 (Stable)
  Amount Paid: PSI=0.0050 (Stable)
  Account_freq: PSI=0.0047 (Stable)
  bank_interaction: PSI=0.0041 (Stable)
  Payment Format: PSI=0.0007 (Stable)
  Account: PSI=0.0001 (Stable)


In [6]:
# Persist monitoring artifacts
monitor_score_path = ARTIFACT_PATH / "monitor_score_shift.csv"
monitor_feature_path = ARTIFACT_PATH / "monitor_feature_shift.csv"
monitor_summary_path = ARTIFACT_PATH / "monitor_summary.json"

score_drift_df.to_csv(monitor_score_path, index=False)

if len(feature_drift) > 0:
    feature_drift.to_csv(monitor_feature_path, index=False)

monitor_summary = {
    "created_at_utc": pd.Timestamp.utcnow().isoformat(),
    "reference_samples": len(risk_ref),
    "new_samples": len(risk_new),
    "score_column_monitored": score_col,
    "score_psi": float(score_drift_df["psi"].iloc[0]),
    "score_interpretation": score_drift_df["interpretation"].iloc[0],
    "features_monitored": len(feature_drift) if len(feature_drift) > 0 else 0,
    "features_with_high_drift": int((feature_drift["psi"] > 0.2).sum()) if len(feature_drift) > 0 else 0,
    "features_with_medium_drift": int(
        ((feature_drift["psi"] >= 0.1) & (feature_drift["psi"] <= 0.2)).sum()
    )
    if len(feature_drift) > 0
    else 0,
    "top_drift_features": feature_drift.head(5)["feature"].tolist() if len(feature_drift) > 0 else [],
}

with open(monitor_summary_path, "w", encoding="utf-8") as f:
    json.dump(monitor_summary, f, indent=2, ensure_ascii=False)

print("Monitoring artifacts saved:")
for path in [monitor_score_path, monitor_feature_path, monitor_summary_path]:
    if path.exists():
        print(f"  - {path.name}")

alerts = []
if monitor_summary["score_psi"] > 0.2:
    alerts.append(
        f"Score drift exceeds threshold (PSI={monitor_summary['score_psi']:.3f})."
    )
if monitor_summary["features_with_high_drift"] > 0:
    alerts.append(
        f"{monitor_summary['features_with_high_drift']} features flagged with high drift."
    )

if alerts:
    print("\nAlerts detected:")
    for alert in alerts:
        print(f"  - {alert}")
else:
    print("\nNo drift alerts triggered.")


Monitoring artifacts saved:
  - monitor_score_shift.csv
  - monitor_feature_shift.csv
  - monitor_summary.json

No drift alerts triggered.


# - Drift Detection

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**OBJETIVO**

Detectar drift nos dados usando Population Stability Index (PSI) em janelas temporais simuladas.

</div>

In [7]:
import json, warnings
import numpy as np, pandas as pd
from pathlib import Path
from scipy.stats import ks_2samp

warnings.filterwarnings("ignore")

# Paths
artifacts_path = Path("../artifacts")
data_path = Path("../data")

print("[OK] Imports carregados")

[OK] Imports carregados


In [8]:
# Carregar dados base
X_base = pd.read_csv(data_path / "X_test_engineered.csv")

# Selecionar features numéricas (top 10 para simplicidade)
numeric_cols = [c for c in X_base.columns if X_base[c].dtype in ['int64', 'float64']]
features = numeric_cols[:10]
X_base = X_base[features]

print(f"[OK] Dados base: {X_base.shape}")
print(f"[OK] Features analisadas: {len(features)}")

[OK] Dados base: (1460, 10)
[OK] Features analisadas: 10


In [9]:
# Simular 4 janelas temporais com drift crescente
np.random.seed(42)
windows = []

for i in range(1, 5):
    # Copiar dados base
    X_window = X_base.copy()
    
    # Aplicar drift artificial (multiplicar por fator crescente)
    drift_factor = 1 + (i * 0.05)  # 5%, 10%, 15%, 20% de drift
    affected_features = features[:min(i+2, len(features))]  # Mais features afetadas a cada janela
    
    for col in affected_features:
        X_window[col] = X_window[col] * drift_factor + np.random.normal(0, 0.01, len(X_window))
    
    windows.append((f"Janela_{i}", X_window, len(affected_features)))
    print(f"[OK] Janela {i}: {len(affected_features)} features com drift {(drift_factor-1)*100:.0f}%")

print(f"\n[OK] Criadas {len(windows)} janelas temporais")

[OK] Janela 1: 3 features com drift 5%
[OK] Janela 2: 4 features com drift 10%
[OK] Janela 3: 5 features com drift 15%
[OK] Janela 4: 6 features com drift 20%

[OK] Criadas 4 janelas temporais


In [10]:
# Calcular PSI simples usando KS test
def simple_psi(baseline, current, bins=10):
    """PSI simplificado usando distribuições binned"""
    try:
        # Criar bins baseados no baseline
        _, bin_edges = np.histogram(baseline, bins=bins)
        
        # Contar frequências em cada bin
        baseline_counts, _ = np.histogram(baseline, bins=bin_edges)
        current_counts, _ = np.histogram(current, bins=bin_edges)
        
        # Normalizar (evitar zeros)
        baseline_pct = (baseline_counts + 1) / (len(baseline) + bins)
        current_pct = (current_counts + 1) / (len(current) + bins)
        
        # Calcular PSI
        psi = np.sum((current_pct - baseline_pct) * np.log(current_pct / baseline_pct))
        return psi
    except:
        return 0

# Calcular PSI para cada feature em cada janela
results = []
for window_name, X_window, n_affected in windows:
    window_psi = []
    for feature in features:
        psi = simple_psi(X_base[feature], X_window[feature])
        window_psi.append(psi)
        results.append({
            'window': window_name,
            'feature': feature, 
            'psi': psi,
            'status': 'Crítico' if psi >= 0.25 else 'Moderado' if psi >= 0.10 else 'Estável'
        })
    
    avg_psi = np.mean(window_psi)
    print(f"{window_name}: PSI médio = {avg_psi:.4f}")

psi_df = pd.DataFrame(results)
psi_df.head()

Janela_1: PSI médio = 0.1571
Janela_2: PSI médio = 0.5149
Janela_3: PSI médio = 1.9229
Janela_4: PSI médio = 1.7803


,window,feature,psi,status
0,Janela_1,Dest Account,0.279241,Crítico
1,Janela_1,Payment Format,1.099647,Crítico
2,Janela_1,From Bank,0.192064,Moderado
3,Janela_1,Account,0.000000,Estável
4,Janela_1,Day,0.000000,Estável


In [11]:
# Resumo por janela
summary = psi_df.groupby('window').agg({
    'psi': ['mean', 'max', 'count'],
    'status': lambda x: (x != 'Estável').sum()
}).round(4)

summary.columns = ['psi_medio', 'psi_max', 'total_features', 'features_com_drift']
summary = summary.reset_index()

print("=== RESUMO DRIFT POR JANELA ===")
for _, row in summary.iterrows():
    print(f"{row['window']}: PSI={row['psi_medio']:.3f}, Max={row['psi_max']:.3f}, Drift={row['features_com_drift']}/{row['total_features']}")

# Features mais problemáticas
print("\n=== TOP 5 FEATURES COM MAIS DRIFT ===")
feature_drift = psi_df.groupby('feature')['psi'].mean().sort_values(ascending=False)
for i, (feature, avg_psi) in enumerate(feature_drift.head().items(), 1):
    print(f"{i}. {feature}: {avg_psi:.4f}")

summary

=== RESUMO DRIFT POR JANELA ===
Janela_1: PSI=0.157, Max=1.100, Drift=3/10
Janela_2: PSI=0.515, Max=4.040, Drift=4/10
Janela_3: PSI=1.923, Max=13.740, Drift=5/10
Janela_4: PSI=1.780, Max=13.731, Drift=6/10

=== TOP 5 FEATURES COM MAIS DRIFT ===
1. Day: 6.8678
2. Payment Format: 2.8991
3. From Bank: 0.5837
4. Dest Account: 0.3296
5. Account: 0.1998


,window,psi_medio,psi_max,total_features,features_com_drift
0,Janela_1,0.1571,1.0996,10,3
1,Janela_2,0.5149,4.0405,10,4
2,Janela_3,1.9229,13.7402,10,5
3,Janela_4,1.7803,13.7310,10,6


In [12]:
# Salvar resultados
artifacts_path.mkdir(exist_ok=True)

# CSV resumido
summary.to_csv(artifacts_path / "drift_detection_summary.csv", index=False)

# Metadata essencial
metadata = {
    "timestamp": pd.Timestamp.now().isoformat(),
    "baseline_samples": len(X_base),
    "features_analyzed": len(features),
    "windows_analyzed": len(windows),
    "drift_summary": {
        "critical_windows": int((summary['psi_medio'] >= 0.25).sum()),
        "moderate_windows": int(((summary['psi_medio'] >= 0.10) & (summary['psi_medio'] < 0.25)).sum()),
        "stable_windows": int((summary['psi_medio'] < 0.10).sum())
    },
    "top_drift_features": feature_drift.head(3).to_dict(),
    "recommendation": "Monitorar features com PSI > 0.25" if feature_drift.max() > 0.25 else "Drift sob controle"
}

with open(artifacts_path / "drift_detection_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("[OK] Artefatos salvos:")
print("- drift_detection_summary.csv")
print("- drift_detection_metadata.json")

# Status final
critical_count = (psi_df['psi'] >= 0.25).sum()
if critical_count > 0:
    print(f"\n[!] ALERTA: {critical_count} casos críticos detectados")
else:
    print(f"\n[OK] ESTÁVEL: Drift dentro do esperado")

print(f" Recomendação: {metadata['recommendation']}")

[OK] Artefatos salvos:
- drift_detection_summary.csv
- drift_detection_metadata.json

[!] ALERTA: 16 casos críticos detectados
 Recomendação: Monitorar features com PSI > 0.25
